In [1]:
import pandas as pd
import numpy as np

metadataDF = pd.read_csv('TheMoviesDataset/movies_metadata.csv', usecols=["id", "title", "budget", "genres", "popularity", "release_date", "revenue", "runtime", "vote_average", "vote_count"])
metadataDF['id'] = metadataDF['id'].astype('int64')
print("Headers: " + str(list(metadataDF.columns.values)))
print(metadataDF.head(2))



Headers: ['budget', 'genres', 'id', 'popularity', 'release_date', 'revenue', 'runtime', 'title', 'vote_average', 'vote_count']
     budget                                             genres    id  \
0  30000000  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   862   
1  65000000  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...  8844   

   popularity release_date    revenue  runtime      title  vote_average  \
0   21.946943   1995-10-30  373554033     81.0  Toy Story           7.7   
1   17.015539   1995-12-15  262797249    104.0    Jumanji           6.9   

   vote_count  
0        5415  
1        2413  


In [ ]:
#Count genre occurences
import ast
from collections import Counter

#Copy DF for manipulation
df_copy = metadataDF.copy()

#Convert the list representations of genres into actual lists.
df_copy['genres'] = df_copy['genres'].apply(ast.literal_eval)
all_genres = [genre['name'] for sublist in df_copy['genres'] for genre in sublist]
genre_counts = Counter(all_genres)
genre_counts_df = pd.DataFrame(genre_counts.items(), columns=['Genre', 'Count'])
print(genre_counts_df.sort_values(by=['Count'], ascending=False))

              Genre  Count
6             Drama  20265
1            Comedy  13182
9          Thriller   7624
5           Romance   6735
7            Action   6596
10           Horror   4673
8             Crime   4307
17      Documentary   3932
3         Adventure   3496
12  Science Fiction   3049
2            Family   2770
13          Mystery   2467
4           Fantasy   2313
0         Animation   1935
15          Foreign   1622
16            Music   1598
11          History   1398
14              War   1323
18          Western   1042
19         TV Movie    767


In [21]:
#Show mean vote_average and mean popularity per genre
#Copy DF for manipulation
df_copy = metadataDF.copy()

#Convert the list representations of genres into actual lists.
df_copy['genres'] = df_copy['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

df_exploded_genres = df_copy.explode('genres')

#Extract genre name and drop original genre column
df_exploded_genres['genre_name'] = df_exploded_genres['genres'].apply(lambda x: x['name'] if isinstance(x,dict) else None)
df_exploded_genres = df_exploded_genres.drop(columns=['genres'])
#Drop rows where 'genre_name' is None
df_exploded_genres = df_exploded_genres.dropna(subset=['genre_name'])

#Ensure vote_average and popularity are numeric
df_exploded_genres['vote_average'] = pd.to_numeric(df_exploded_genres['vote_average'], errors='coerce')
df_exploded_genres['popularity'] = pd.to_numeric(df_exploded_genres['popularity'], errors='coerce')

#Drop rows where 'vote_average' or 'popularity' is NaN
df_exploded_genres = df_exploded_genres.dropna(subset=['vote_average', 'popularity'])

genre_stats = df_exploded_genres.groupby('genre_name').agg({
    'vote_average': 'mean',
    'popularity': 'mean'
}).reset_index()

print(genre_stats[['genre_name', 'vote_average']].sort_values(by=['vote_average'], ascending=False))
print("\n")
print(genre_stats[['genre_name', 'popularity']].sort_values(by=['popularity'], ascending=False))

         genre_name  vote_average
2         Animation      6.275556
10          History      6.154220
18              War      6.041119
6             Drama      5.905226
12            Music      5.879599
4             Crime      5.878407
5       Documentary      5.823093
13          Mystery      5.820470
14          Romance      5.802658
8           Fantasy      5.787722
7            Family      5.753394
9           Foreign      5.737978
3            Comedy      5.715741
1         Adventure      5.686070
17         Thriller      5.616763
0            Action      5.584036
15  Science Fiction      5.343096
16         TV Movie      5.235072
11           Horror      5.230195
19          Western      5.167179


         genre_name  popularity
1         Adventure    5.998335
8           Fantasy    5.363656
15  Science Fiction    4.997888
0            Action    4.770506
7            Family    4.729328
2         Animation    4.710670
17         Thriller    4.487044
4             Crime    4.144

In [28]:
#Create creditsDF dataframe
creditsDF = pd.read_csv('TheMoviesDataset/credits.csv')

print("Headers: " + str(list(creditsDF.columns.values)))
print(creditsDF.head(2))

Headers: ['cast', 'crew', 'id']
                                                cast  \
0  [{'cast_id': 14, 'character': 'Woody (voice)',...   
1  [{'cast_id': 1, 'character': 'Alan Parrish', '...   

                                                crew    id  
0  [{'credit_id': '52fe4284c3a36847f8024f49', 'de...   862  
1  [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...  8844  


In [31]:
#Show cast statistics
metadata_join_credits_DF = metadataDF.merge(creditsDF, on='id', how='inner')

#Copy DF for manipulation
df_copy = metadata_join_credits_DF.copy()

#Convert the list representations of genres into actual lists.
df_copy['cast'] = df_copy['cast'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

df_exploded_cast = df_copy.explode('cast')

#Extract cast name and drop original cast column
df_exploded_cast['cast_name'] = df_exploded_cast['cast'].apply(lambda x: x['name'] if isinstance(x,dict) else None)
df_exploded_cast = df_exploded_cast.drop(columns=['cast'])
#Drop rows where 'cast_name' is None
df_exploded_cast = df_exploded_cast.dropna(subset=['cast_name'])

#Ensure vote_average and popularity are numeric
df_exploded_cast['vote_average'] = pd.to_numeric(df_exploded_cast['vote_average'], errors='coerce')
df_exploded_cast['popularity'] = pd.to_numeric(df_exploded_cast['popularity'], errors='coerce')

#Drop rows where 'vote_average' or 'popularity' is NaN
df_exploded_cast = df_exploded_cast.dropna(subset=['vote_average', 'popularity'])

cast_stats = df_exploded_cast.groupby('cast_name').agg({
    'vote_average': 'mean',
    'popularity': 'mean',
    'id': 'count' #Count of films they have been in
}).reset_index()

cast_stats = cast_stats.rename(columns={'id': 'num_films'})
print(cast_stats[['cast_name', 'num_films', 'vote_average']].sort_values(by=['vote_average'], ascending=False))
print("\n")
print(cast_stats[['cast_name', 'num_films', 'popularity']].sort_values(by=['popularity'], ascending=False))
print(cast_stats[cast_stats['cast_name'] == 'Tom Hanks'])
print(cast_stats[cast_stats['num_films'] == 241])

                   cast_name  num_films  vote_average
76731              Idris Ali          1          10.0
52524            Elif Baysal          1          10.0
2899            Al Mackenzie          1          10.0
34789        Chuck Blackwell          1          10.0
147557          Pamela Craig          1          10.0
...                      ...        ...           ...
113019         Leo Bruckmann          1           0.0
12586              Anni Timm          1           0.0
181395          Sven Jerring          1           0.0
146781       Oscar Tengström          1           0.0
24       Rosenda Schaschmidt          1           0.0

[202747 rows x 3 columns]


                 cast_name  num_films  popularity
4726          Alex Dowding          1  547.488298
173537       Shaun Newnham          1  294.337037
60652   Frank Allen Forbes          1  294.337037
57188        Eva Dabrowski          1  294.337037
179026         Steve Doyle          1  294.337037
...                    

In [32]:
#Show correlation between how many movies actors are in and how well those movies are received
numFilmsCorrelation = cast_stats.groupby('num_films').agg({
    'vote_average': 'mean',
    'popularity': 'mean'
}).reset_index()

print(numFilmsCorrelation)

     num_films  vote_average  popularity
0            1      5.880034    5.169291
1            2      5.873058    5.313597
2            3      5.885797    5.195781
3            4      5.859514    4.925731
4            5      5.843277    4.742350
..         ...           ...         ...
103        110      6.102273    5.975370
104        123      6.211382   11.706544
105        125      5.666400    3.092939
106        148      5.870270    4.749606
107        241      5.539004    2.028024

[108 rows x 3 columns]
